<a href="https://colab.research.google.com/github/thant-san/llm-zoomcamp/blob/main/02_homework(LLM).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install fastembed

In [ ]:
from fastembed import TextEmbedding
import numpy as np

In [ ]:
query = "I just discovered the course. Can I join now?"
model_name = "jinaai/jina-embeddings-v2-small-en"
embedder = TextEmbedding(model_name=model_name)
query_embedding = list(embedder.embed([query]))[0]

print(len(query_embedding))
print("Q1 - Min value in query vector:", np.min(query_embedding))

In [ ]:
q=query_embedding
np.linalg.norm(q)

In [ ]:
q.dot(q)

In [ ]:
doc = 'Can I still join the course after the start date?'

doc_embedder = TextEmbedding(model_name=model_name)
doc_embedding = list(embedder.embed([doc]))[0]
cos_sim = np.dot(query_embedding, doc_embedding)
print("Q2 - Cosine similarity:", round(cos_sim, 2))

In [ ]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [ ]:
doc_texts = [doc["text"] for doc in documents]
doc_vectors = list(embedder.embed(doc_texts))
scores_q3 = [np.dot(query_embedding, vec) for vec in doc_vectors]
print("Q3 - Best match index:", np.argmax(scores_q3))

In [ ]:
full_texts = [doc["question"] + " " + doc["text"] for doc in documents]
full_vectors = list(embedder.embed(full_texts))
scores_q4 = [np.dot(query_embedding, vec) for vec in full_vectors]
print("Q4 - Best match index:", np.argmax(scores_q4))

In [ ]:
embedder = TextEmbedding(model_name="BAAI/bge-small-en")
vec = list(embedder.embed(["test sentence"]))[0]
print(len(vec))

In [ ]:
!pip install fastembed qdrant-client



In [ ]:
from fastembed import TextEmbedding
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PointStruct
import requests
import uuid

In [ ]:
docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [ ]:
# Prepare the text: concatenate question + text
full_texts = [doc['question'] + ' ' + doc['text'] for doc in documents]

In [ ]:
# Embed using BAAI/bge-small-en (384 dim)
embedder = TextEmbedding(model_name="BAAI/bge-small-en")
embeddings = list(embedder.embed(full_texts))

In [ ]:
# Create a collection
client = QdrantClient(path="qdrant_index")  # local file-based
collection_name = "ml-zoomcamp-faq"
client.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=384, distance=Distance.COSINE),
)

# Upload vectors
points = [
    PointStruct(
        id=str(uuid.uuid4()),
        vector=embedding,
        payload={"text": full_texts[i], "original": documents[i]}
    )
    for i, embedding in enumerate(embeddings)
]

client.upsert(collection_name=collection_name, points=points)

# Q6 Query
query = "I just discovered the course. Can I join now?"
query_vector = list(embedder.embed([query]))[0]

search_result = client.search(
    collection_name=collection_name,
    query_vector=query_vector,
    limit=1,
)

# Print top result and its score
top_hit = search_result[0]
print("Q6 - Top score:", round(top_hit.score, 2))
print("Matched question:", top_hit.payload['original']['question'])